## Images (converted to bmp) are currently just randomly names 0-9, with no labelling in regards to exposure. Going to use the max intesnoty value to rename the images according to exposure and camera settings e.g. 2GyAsecondauto1000.bmp


### Read in files from the bmp directory

In [68]:
import glob
import os
import sys
import numpy as np
import scipy.ndimage as scimg
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
now = datetime.datetime.now()
print("Date and time ",str(now))

############################
#
dose_key = 'ControlAsecond'
contrast_key = 'full'
dir_key = dose_key + '/' + contrast_key
root_dir = '../CometImages/CA210819/Collated/' + dir_key + '/bmp/'
image_dir = root_dir
#
image_dir = root_dir + '*.bmp'  #where the image directory is - find all bmp files here

print("image_dir is: ", image_dir)
fileList= sorted(glob.glob(image_dir))  #whole file path, as set in above line
print("Files read in (whole path) are", fileList)
#
fileNames = [os.path.basename(x) for x in sorted(glob.glob(image_dir))]  #only file name with extension
print("\nFile names (with extension) are:", fileNames)
#
nFiles = len(fileNames)
print("\n",nFiles, " files will be analysed")
#
fileNamesSplit = np.empty((nFiles), dtype = 'object')
for n in range (0, nFiles):
    x = fileNames[n]
    fileNamesSplit[n] = os.path.splitext(x)[0]   #remove extension from file name
print("\nfileNamesSplit is: ", fileNamesSplit)
#
#
then = now
now = datetime.datetime.now()
print(" ")
print("Date and time",str(now))
print("Time since last check is",str(now - then))

Date and time  2021-09-01 15:56:46.971899
image_dir is:  ../CometImages/CA210819/Collated/ControlAsecond/full/bmp/*.bmp
Files read in (whole path) are ['../CometImages/CA210819/Collated/ControlAsecond/full/bmp/000.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/001.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/002.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/003.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/004.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/005.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/006.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/007.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/008.bmp', '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/009.bmp']

File names (with extension) are: ['000.bmp', '001.bmp', '002.bmp', '003.bmp', '004.bmp', '005.bmp', '006.bmp', '007.bmp', '008.bmp', '009.bmp']

 10  files will be ana

### Find min and max intesnsities of each image and write to array. Order array ascending order according to max intensity (to order the file names) and then rename as Dose,position,contrast,exposure

In [70]:
import os
import datetime
now = datetime.datetime.now()
print("Date and time ",str(now))
#
imgIntensities = np.empty((nFiles, 3), dtype = 'object')
#
for n in range(0, nFiles):
    imFile = fileList[n]
    fileName = fileNames[n]
    #
    imgRaw = plt.imread(imFile)
    shape = imgRaw.shape
    print("shape is: ", shape)
    nRows = imgRaw.shape[0] # y coordinates
    nCols = imgRaw.shape[1] # x ccordinates
    if len(shape) == 3:
        nDepth = imgRaw.shape[2]
    else:
        nDepth = 1
    # Determine image format and process accordingly
    if nDepth == 4:
        imgRGB = rgba2rgb(imgRaw)
        imgGrey = rgb2grey(imgRGB, 1)
    elif nDepth == 3:
        imgRGB = imgRaw
        imgGrey = rgb2grey(imgRGB, 1)
    elif nDepth == 1:
        imgRGB = imgRaw
        imgGrey = imgRaw
    else:
        print(" ")
        print("Unexpected image depth",nDepth)
        sys.stop()
    imgArr = np.zeros((nRows, nCols))
    imgArr[:, :] = imgGrey[:, :]
    #
    maxPix = np.amax(imgArr)
    minPix = np.amin(imgArr)
    print("\nType of raw image file is",imgRaw.dtype)
    print("Type of image file is",imgArr.dtype)
    print("\nNumber of rows",nRows,"of columns",nCols,"of pixels",nRows*nCols,"and depth",nDepth)
    print("\nMaximum value intensity is: ",maxPix)
    print("\nMinimum value intensity is: ",minPix)
    #
    #write values to intensity array
    imgIntensities[n, 0] = fileName
    imgIntensities[n, 1] = minPix
    imgIntensities[n, 2] = maxPix
    #
print("")
print("original intensities array is: ", imgIntensities)
#

#sort imgIntesnities by max intesnity ascending
imgIntAsc = imgIntensities[np.argsort(imgIntensities[:, 2])]
print("\nsorted intensity array is: \n", imgIntAsc)
#
#make array of all exposure values
exposures = np.linspace(1000, 10000, 10, dtype= int)
#exposures = np.linspace(500, 3000, 6, dtype = int)
print("exposure array: ", exposures)
#
#loop for renaming all bmp files in image_dir according to the sorted intensity array in previous cell
for n in range(0, nFiles): 
    old_path = root_dir + imgIntAsc[n, 0]
    new_path = root_dir + dose_key + contrast_key + str(exposures[n]) + ".bmp"
    print("old path is: ", old_path)
    print("new path is: ", new_path)
    os.rename(old_path, new_path)
#
then = now
now = datetime.datetime.now()
print(" ")
print("Date and time",str(now))
print("Time since last check is",str(now - then))

Date and time  2021-09-01 16:38:23.566768
shape of imgIntesnities is:  (10, 3)


FileNotFoundError: [Errno 2] No such file or directory: '../CometImages/CA210819/Collated/ControlAsecond/full/bmp/000.bmp'